In [13]:
%%bash
pip install pydicom opencv-python scikit-image
pip install pyradiomics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 33.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Discarding https://files.pythonhosted.org/packages/03/c1/20fc2c50ab1e3304da36d866042a1905a2b05a1431ece35448ab6b4578f2/pyradiomics-3.1.0.tar.gz (from https://pypi.org/simple/pyradiomics/): Requested pyradiomics from https://files.pythonhosted.org/packages/03/c1/20fc2c50ab1e3304da36d866042a1905a2b05a1431ece35448ab6b4578f2/pyradiomics-3.1.0.tar.gz has inconsistent version: expected '3.1.0', but metadata 

In [10]:
%%bash
wget --show-progress http://www.inf.ufpr.br/lferrari/imagens_ihq.tar.gz


--2023-11-19 23:38:03--  http://www.inf.ufpr.br/lferrari/imagens_ihq.tar.gz
Resolving www.inf.ufpr.br (www.inf.ufpr.br)... 200.17.202.113, 2801:82:80ff:8001:216:ccff:feaa:79
Connecting to www.inf.ufpr.br (www.inf.ufpr.br)|200.17.202.113|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.inf.ufpr.br/lferrari/imagens_ihq.tar.gz [following]
--2023-11-19 23:38:04--  https://www.inf.ufpr.br/lferrari/imagens_ihq.tar.gz
Connecting to www.inf.ufpr.br (www.inf.ufpr.br)|200.17.202.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74889388 (71M) [application/octet-stream]
Saving to: ‘imagens_ihq.tar.gz.1’

     0K .......... .......... .......... .......... ..........  0%  406K 3m0s
    50K .......... .......... .......... .......... ..........  0%  405K 3m0s
   100K .......... .......... .......... .......... ..........  0% 1.26M 2m19s
   150K .......... .......... .......... .......... ..........  0%  588K 2m15s
   2

CalledProcessError: ignored

In [12]:
!tar -xvf imagens_ihq.tar.gz


imagens_ihq_er/
imagens_ihq_er/0/
imagens_ihq_er/0/864RE_s1c0x101924-1600y53572-1200m1348.png
imagens_ihq_er/0/785RE_s4c0x194287-1600y67928-1200m286.png
imagens_ihq_er/0/331RE_s3c0x144885-1600y184297-1200m16814.png
imagens_ihq_er/0/095RE_s1c0x120577-1600y62114-1200m3452.png
imagens_ihq_er/0/785RE_s4c0x182875-1600y75453-1200m554.png
imagens_ihq_er/0/785RE_s0c0x163362-1600y24598-1200m1064.png
imagens_ihq_er/0/864RE_s1c0x100550-1600y76100-1200m1981.png
imagens_ihq_er/0/203RE_s1c0x126403-1600y139278-1200m8366.png
imagens_ihq_er/0/020RE_s3c0x191243-1600y77354-1200m2084.png
imagens_ihq_er/0/056RE_s2c0x271949-1600y123227-1200m6802.png
imagens_ihq_er/0/203RE_s1c0x70680-1600y21431-1200m807.png
imagens_ihq_er/0/058RE_s1c0x83385-1600y116327-1200m1638.png
imagens_ihq_er/0/224RE_s1c0x164123-1600y20492-1200m89.png
imagens_ihq_er/0/203RE_s1c0x156431-1600y72864-1200m4070.png
imagens_ihq_er/0/095RE_s1c0x167711-1600y117860-1200m7586.png
imagens_ihq_er/0/638RE_s2c0x73311-1600y184334-1200m893.png
imagens_

In [14]:
import cv2 as cv
import numpy as np
import os
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from joblib import dump, load
import pydicom as dicom
import radiomics
from radiomics import featureextractor
import SimpleITK as sitk
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt

In [21]:
def cut_images(input_path, output_path, new_width, new_height):
    """
    Cut images into the desired size and save the output images

    Params:
    input_path = path to the original images
    output_path = path to save the cut images
    new_width = width of the cut images
    new_height = height of the cut images

    Return:
    None
    """

    # Create dir if it doesn't exist
    os.makedirs(output_path, exist_ok=True)

    n_images = 0

    # Browse input path
    for class_dir in os.listdir(input_path):
        class_path = os.path.join(input_path, class_dir)

        # If it is a directory
        if os.path.isdir(class_path):

            os.makedirs(os.path.join(output_path, class_dir), exist_ok=True)

            # Save image id
            image_id = 1

            # Go through images
            for image_file in os.listdir(class_path):
                image_path = os.path.join(class_path, image_file)

                # Save patient id
                patient = image_file.split("_")[0]

                image = cv.imread(image_path)

                # If image exists
                if image is not None:

                    # Save subimage id
                    sub_id = 1

                    for i in range(0, image.shape[0], new_height):
                        for j in range(0, image.shape[1], new_width):

                            # Cut image into subimage
                            sub_image = image[i:i+new_height, j:j+new_width]

                            # Output file path
                            output_file = f"{patient}_img{image_id}-{sub_id}.png"
                            output_file = os.path.join(output_path, class_dir, output_file)

                            # Save subimage
                            cv.imwrite(output_file, sub_image)

                            sub_id += 1
                            n_images += 1

                    image_id += 1


In [5]:
def read_images(input_path):
    """
    Read images in the input_path,
    save image, patient of each image and the class (group/labels)

    Params:
    input_path = path to the original images

    Return:
    images = list of all images
    patients = list with patient id for each image
    classes = list with class for each image
    """

    # Lists to save images, patients and classes
    images = []
    patients = []
    classes = []

    # Browse input path
    for class_dir in os.listdir(input_path):
        class_path = os.path.join(input_path, class_dir)

        # If it is a directory
        if os.path.isdir(class_path):

            for image_file in os.listdir(class_path):
                image_path = os.path.join(class_path, image_file)

                patient = image_file.split("_")[0]

                image = cv.imread(image_path, cv.IMREAD_GRAYSCALE)

                # Append image, patient id and class to list
                images.append(image)
                patients.append(patient)
                classes.append(class_dir)

    return (images, patients, classes)

In [6]:
def divide_folds(images, patients, classes):
    """
    Divides a dataset into folds for stratified k-fold cross-validation.

    Params:
    images = list of all images
    patients = list with patient id for each image
    classes = list with class for each image

    Return:
    folds = list of tuples, each tuple is one folder
    """
    # Create a list of unique indexes for patients
    unique_patients = list(set(patients))

    # Shuffle the list of unique indexes
    random.shuffle(unique_patients)

    # Divide patients into groups
    n_folds = 4 # since it's not an exact division, there will be 5 folds
    fold_size = len(unique_patients) // n_folds
    patients_folds = [unique_patients[i:i+fold_size] for i in range(0, len(unique_patients), fold_size)]

    # List to save folds
    folds = []

    # Divide images into folds based on patients
    for i, patients_folds in enumerate(patients_folds):
        train_patients = [p for p in unique_patients if p not in patients_folds]
        test_patients = patients_folds

        train_indices = [i for i, patient in enumerate(patients) if patient in train_patients]
        test_indices = [i for i, patient in enumerate(patients) if patient in test_patients]

        train_images = [images[i] for i in train_indices]
        test_images = [images[i] for i in test_indices]
        train_classes = [classes[i] for i in train_indices]
        test_classes = [classes[i] for i in test_indices]

        folds.append((train_images, test_images, train_classes, test_classes))

    return folds


In [7]:
def apply_thresholds(imgs):
    """
    Apply Otsu's and Adaptative thresholds to images

    Params:
    imgs = list of raw images

    Return:
    imgs_otsu = Otsu's thresholded images
    imgs_adapt = Adaptative thresholded images
    """

    imgs_otsu = []
    imgs_adapt = []

    # For each image in dataset
    for img in imgs:

        # Otsu's thresholding
        _, th1 = cv.threshold(img, 100, 1, cv.THRESH_BINARY+cv.THRESH_OTSU)

        # Adaptative gaussian thresholding
        #th2 = cv.adaptiveThreshold(img,255,cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY,11,2)
        th3 = cv.adaptiveThreshold(img, 1, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11, 2)

        imgs_otsu.append(th1)
        imgs_adapt.append(th3)

    return (imgs_otsu, imgs_adapt)

In [8]:
def update_folds(folds):
    """
    For each fold, add Otsu's and adaptive
    thresholding of all test and train images

    Params:
    folds = list of tuples containing the images divided in folds

    Returns:
    new_folds = updated list of tuples containing the images and masks divided in folds
    """

    new_folds = []

    # For each fold
    for i in range(len(folds)):
        fold = folds[i]

        # Get train and test images
        x_train = fold[0]
        x_test = fold[1]
        y_train = fold[2]
        y_test = fold[3]

        x_train_otsu, x_train_adapt = apply_thresholds(x_train)

        # Update list with the thresholds
        new_folds.append((x_train, x_train_otsu, x_train_adapt, x_test, y_train, y_test))

    return new_folds

In [22]:
# Cut images into 40x30
cut_images("/content/imagens_ihq_er", "/content/imagens_cortadas", 40, 30)

# Get images, patients and classes from a path
images, patients, classes = read_images("/content/imagens_cortadas")

# Divide images into folds
folds = divide_folds(images, patients, classes)

# Update folds, applying Otsu's and adaptive thresholds
folds = update_folds(folds)

# Save folds locally
#dump(folds, '../folds.joblib')

# Load folds
# folds = load('../folds.joblib')

In [ ]:
# Load folds
folds = load('../folds.joblib')

Extract features with PyRadiomics

In [24]:
def run_extractor(imgs, otsu, adapt, extractor):
    """
    Extract features using sitk and pyradiomics

    Params:
    imgs = raw images
    otsu = masked images with otsu thresholding
    adapt = masked images with adaptative thresholding
    extractor = pyradiomics extractor

    Returns:
    features_otsu = features for otsu mask
    features_adapt = features for adaptative mask
    """

    data_spacing=[1,1,1]
    features_otsu = []
    features_adapt = []

    for idx in range(len(imgs)):

        # Get raw, Otsu's and adaptive images
        img = imgs[idx]
        img_otsu = otsu[idx]
        img_adapt = adapt[idx]

        sitk_img = sitk.GetImageFromArray(img)
        sitk_img.SetSpacing((1, 1, 1))
        sitk_img = sitk.JoinSeries(sitk_img)

        sitk_otsu = sitk.GetImageFromArray(img_otsu)
        sitk_otsu.SetSpacing((1, 1, 1))
        sitk_otsu = sitk.JoinSeries(sitk_otsu)
        sitk_otsu = sitk.Cast(sitk_otsu, sitk.sitkInt32)

        sitk_adapt = sitk.GetImageFromArray(img_adapt)
        sitk_adapt.SetSpacing((1, 1, 1))
        sitk_adapt = sitk.JoinSeries(sitk_adapt)
        sitk_adapt = sitk.Cast(sitk_otsu, sitk.sitkInt32)

        # Extract features and append them to the proper list
        try:
            ft_otsu = extractor.execute(sitk_img, sitk_otsu)
            features_otsu.append(ft_otsu)

            ft_adapt = extractor.execute(sitk_img, sitk_adapt)
            features_adapt.append(ft_adapt)

        except:
            #print(f"{idx}, ", end="")
            pass

    return (features_otsu, features_adapt)



In [27]:
def conditional_append(element, dest):
    """
    Append element to the list destiny, if element is not in destiny

    Params:
    element = an element of any kind
    dest = a destination list

    Returns:
    destiny = list with appended element if the element was not in there
    """
    if element not in dest:
        dest.append(element)

    return dest

def process_features(feats_o, feats_a):
    """
    Process features, in a way that:
    - features that are dictionaries and strings are removed
    - features that are tuples are separated and each element
    of the tuple is considered one feature
    - other types are converted to float

    Params:
    feats_o = list of Otsu's threshold features
    feats_a = list of adaptativa threshold features

    Returns:
    all_feats_o = Otsu's features processed
    all_feats_a = adaptative features processed
    names = feature names processed
    """

    all_feats_o = []
    all_feats_a = []
    names = []

    # For each image in one of the features list
    for sample_id in range(len(feats_o)):

        # Get features for Otsu's and adaptive for this sample
        sample_o = feats_o[sample_id]
        sample_a = feats_a[sample_id]

        values_o = []
        values_a = []

        # For each feature in the list
        for key in sample_o:

            # Get the feature's value
            value_o = sample_o[key]
            value_a = sample_a[key]

            # If the value is str or dict, ignore it
            if type(value_o) == str or type(value_o) == dict:
                continue
            # If it's a tuple
            elif type(value_o) == tuple:
                for e in range(len(value_o)):
                    # Add and index to the feature name
                    conditional_append(f'{key}_{e}', names)
                    # Append float values to the lists
                    values_o.append(float(value_o[e]))
                    values_a.append(float(value_a[e]))
            # For other data types, just append the name and float values
            else:
                conditional_append(key, names)
                values_o.append(float(value_o))
                values_a.append(float(value_a))

        # Append processed features to the general list
        all_feats_o.append(values_o)
        all_feats_a.append(values_a)

    return (all_feats_o, all_feats_a, names)

def extract_features(folds):
    """
    Process features, in a way that:
    - features that are dictionaries and strings are removed
    - features that are tuples are separated and each element
    of the tuple is considered one feature
    - other types are converted to float
    Get the features' names, with tuple features indexed

    Params:
    folds = list of tuples containing x_train raw and with thresholds

    Returns:
    all_folds_feats = dictionary containing Otsu's features and adaptive
    features for each fold
    names = feature names
    """

    # Create feature extractor
    !wget -c https://raw.githubusercontent.com/AIM-Harvard/pyradiomics/master/examples/exampleSettings/Params.yaml
    params = 'Params.yaml'
    settings = {'label': 1, 'correctMask': True}
    extractor = featureextractor.RadiomicsFeatureExtractor(params, additionalInfo=True, **settings)

    # Dictionary to keep all features for all folds
    all_folds_feats = {}

    # For each fold
    for fold_id in range(len(folds)):

        fold = folds[fold_id]

        # Get x_train and threshold images
        x_train = fold[0]
        x_train_otsu = fold[1]
        x_train_adapt = fold[2]

        # Extract features from Otsu's and adaptative
        feats_o, feats_a = run_extractor(x_train, x_train_otsu, x_train_adapt, extractor)

        # Process features and get feature names
        all_feats_o, all_feats_a, names = process_features(feats_o, feats_a)

    all_folds_feats[fold_id+1] = (all_feats_o, all_feats_a)

    return (all_folds_feats, names)

In [ ]:
all_folds_feats, names = extract_features(folds)

--2023-11-19 23:46:29--  https://raw.githubusercontent.com/AIM-Harvard/pyradiomics/master/examples/exampleSettings/Params.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3037 (3.0K) [text/plain]
Saving to: ‘Params.yaml’

Params.yaml         100%[===================>]   2.97K  --.-KB/s    in 0s      

2023-11-19 23:46:29 (37.5 MB/s) - ‘Params.yaml’ saved [3037/3037]



Streaming output truncated to the last 5000 lines.
INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask
INFO:radiomics.featureextractor:Computing shape
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Computing firstorder
INFO:radiomics.featureextractor:Computing glcm
INFO:radiomics.featureextractor:Computing glrlm
INFO:radiomics.featureextractor:Computing glszm
INFO:radiomics.featureextractor:Computing gldm
INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask
INFO:radiomics.featureextractor:Computing shape
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Computing firstord

In [ ]:
import csv
for i in range(len(all_folds_feats)):
    feat_o = all_folds_feats[i][0]
    feat_a = all_folds_feats[i][1]
    filename_o = f'fold{i+1}_o.csv'

    filename_a = f'fold{i+1}_a.csv'

    with open(filename_o, 'w', newline='') as f:
    # Step 4: Using csv.writer to write the list to the CSV file
        writer = csv.writer(f)
        writer.writerows(feat_o) # Use writerow for single list

    with open(filename_a, 'w', newline='') as f:
    # Step 4: Using csv.writer to write the list to the CSV file
        writer = csv.writer(f)
        writer.writerows(feat_a) # Use writerow for single list

In [ ]:
with open('names.txt', 'w') as f:
    f.write(names)

In [ ]:
len(all_folds_feats[1][0][0])


129